In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 5);
sns.set_style('whitegrid')
import matplotlib.colors as mcolors
!pip install geopandas
import geopandas as gpd
import folium
from folium.plugins import HeatMap
import os
import plotly.express as px 
import plotly.graph_objs as go
from plotly.offline import iplot
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=False)
from geopy.geocoders import Nominatim
! pip install dexplot
import dexplot as dxp
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/kaggle/input/zomato-bangalore-restaurants/zomato.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
((df.isnull() | df.isna()).sum()*100 / df.index.size)

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df['menu_item'].unique()

In [ ]:
sns.countplot(df['book_table'])
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Restaurants provides Table Booking or Not')

In [ ]:
sns.countplot(df['online_order'])
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Restaurants delivering online or Not')

In [ ]:
df['listed_in(type)'].unique()

In [ ]:
dxp.count(val='listed_in(type)', data=df, figsize=(20,12), split='listed_in(city)', normalize=True)

In [ ]:
print(df['rest_type'].nunique())
df['rest_type'].unique()

In [ ]:
plt.figure(figsize=(12,8))
val = df['name'].value_counts()[:25]  
sns.barplot(x=val, y=val.index, palette = "Set1")
plt.title("Famous Restaurants in the City of Bengaluru")
plt.xlabel("Number of Outlets")

In [ ]:
df['rate'].unique()

In [ ]:
df.rate = df.rate.replace("NEW", np.nan)
df.dropna(how ='any', inplace = True)

In [ ]:
data = df 
data['rate'] = data['rate'].astype(str) 
data['rate'] = data['rate'].apply(lambda x: x.replace('/5',''))
data['rate'] = data['rate'].apply(lambda x: float(x)) 
data.head()

In [ ]:
rel_type = pd.crosstab(df['rate'], df['listed_in(city)'])
rel_type.plot(kind='bar', stacked=True, figsize=(20,16))
plt.title('City - Rating',fontsize=18)
plt.ylabel('City',fontsize=12)
plt.xlabel('Rating',fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

In [ ]:
plt.figure(figsize=(12,8))
sns.distplot(df['rate'], bins = 25)

In [ ]:
sns.countplot(df['listed_in(type)'])
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.title('Restaurants delivering online or Not')

In [ ]:
locations = pd.DataFrame({"Name": df['location'].unique()})
locations.head()

In [ ]:
locations['Name']=locations['Name'].apply(lambda x: "Bangaluru " + str(x)) # here I have used lamda function
lat_lon=[]
geolocator=Nominatim(user_agent="app")
for location in locations['Name']:
    location = geolocator.geocode(location)
    if location is None:
        lat_lon.append(np.nan)
    else:    
        geo=(location.latitude,location.longitude)
        lat_lon.append(geo)

locations['geo_loc']=lat_lon
locations.to_csv('locations.csv',index=False)

In [ ]:
locations["Name"]=locations['Name'].apply(lambda x :  x.replace("Bangaluru","")[1:])
locations.head()

In [ ]:
def generateBaseMap(default_location=[12.97, 77.59], default_zoom_start=12):
  base_map = folium.Map(location = default_location, control_scale=True, zoom_start=default_zoom_start)
  return base_map

In [ ]:
Rest_locations = pd.DataFrame(df['location'].value_counts().reset_index())
Rest_locations.columns = ['Name', 'count']
Rest_locations = Rest_locations.merge(locations,on='Name', how="left").dropna()

In [ ]:
Rest_locations

In [ ]:
lat,lon=zip(*np.array(Rest_locations['geo_loc']))
Rest_locations['lat']=lat
Rest_locations['lon']=lon
basemap=generateBaseMap()
HeatMap(Rest_locations[['lat','lon','count']].values.tolist(),radius=15).add_to(basemap)

In [ ]:
basemap

In [ ]:
def produce_data(col,name):
    data= pd.DataFrame(df[df[col]==name].groupby(['location'],as_index=False)['url'].agg('count'))
    data.columns=['Name','count']
    print(data.head())
    data=data.merge(locations,on="Name",how='left').dropna()
    data['lan'],data['lon']=zip(*data['geo_loc'].values)
    return data.drop(['geo_loc'],axis=1)

In [ ]:
food=produce_data('cuisines','South Indian')
basemap=generateBaseMap()
HeatMap(food[['lan','lon','count']].values.tolist(),radius=15).add_to(basemap)
basemap

In [ ]:
food=produce_data('cuisines','North Indian')
basemap=generateBaseMap()
HeatMap(food[['lan','lon','count']].values.tolist(),radius=15).add_to(basemap)
basemap

In [ ]:
df.info()

In [ ]:
df.online_order.replace(('Yes','No'),(True, False),inplace=True)
df.book_table.replace(('Yes','No'),(True, False),inplace=True)

In [ ]:
def Encode(df):
    for column in df.columns[~df.columns.isin(['rate', 'approx_cost(for two people)', 'votes'])]:
        df[column] = df[column].factorize()[0]
    return df

df_en = Encode(df.copy())

In [ ]:
df_en.info()

In [ ]:
df_en.head(50)

In [ ]:
df_en['approx_cost(for two people)'] = df_en['approx_cost(for two people)'].astype(str)
df_en['approx_cost(for two people)'] = df_en['approx_cost(for two people)'].apply(lambda x: x.replace(',','.'))
df_en['approx_cost(for two people)'] = df_en['approx_cost(for two people)'].astype(float)

In [ ]:
df_en.info()

In [ ]:
corr = df_en.corr()
plt.figure(figsize=(15,8))
sns.heatmap(corr, annot=True)
df_en.columns

In [ ]:
from sklearn.model_selection import train_test_split
x=df_en.drop(["rate"], axis=1) #modified
y = df_en['rate']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42)
x_train.head()

In [ ]:
y_train.head()

In [ ]:
x_test.head()

In [ ]:
y_test.head()

In [ ]:
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
reg_model = LinearRegression() 
reg_model.fit(x_train, y_train)
y_pred = reg_model.predict(x_test) 
r2_score(y_test, y_pred) 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor(n_estimators=600, criterion='mse', max_depth=None
                                 , min_samples_split=2, min_samples_leaf=0.0001, min_weight_fraction_leaf=0.0,
                                 max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0,
                                 min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None,
                                 random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)
RF_model.fit(x_train, y_train)
y_predict = RF_model.predict(x_test) 
r2_score(y_test, y_predict)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
DT_model = DecisionTreeRegressor(criterion='mse', splitter='best', max_depth=None,
                                 min_samples_split=2, min_samples_leaf=0.00011, min_weight_fraction_leaf=0.0,
                                 max_features=None, random_state=None, max_leaf_nodes=None,
                                 min_impurity_decrease=0.0, min_impurity_split=None,
                                 presort='deprecated', ccp_alpha=0.0)
DT_model.fit(x_train, y_train) 
y_predict = DT_model.predict(x_test)
r2_score(y_test, y_predict)